In [14]:
"""
Performance Impact Analysis
Compares augmented vs non-augmented training on multiple descriptors and replicates
"""

import os
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
from tqdm import tqdm
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import xgboost as xgb
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors, rdMolDescriptors
from rdkit.Avalon import pyAvalonTools
import sys

# Add parent directory to path for MMPDataAugmentor import
sys.path.append('..')

warnings.filterwarnings('ignore')

# ============================================================================
# Feature Generation Functions
# ============================================================================

def generate_avalon_features(smiles_list, n_bits=2048):
    """Generate Avalon fingerprints"""
    features = []
    valid_indices = []
    
    for idx, smiles in enumerate(smiles_list):
        mol = Chem.MolFromSmiles(smiles)
        if mol is not None:
            fp = pyAvalonTools.GetAvalonFP(mol, nBits=n_bits)
            features.append(np.array(fp))
            valid_indices.append(idx)
    
    return np.array(features), valid_indices

def generate_rdkit2d_features(smiles_list):
    """Generate RDKit 2D descriptors"""
    features = []
    valid_indices = []
    
    descriptor_names = [desc[0] for desc in Descriptors._descList]
    
    for idx, smiles in enumerate(smiles_list):
        mol = Chem.MolFromSmiles(smiles)
        if mol is not None:
            desc_values = []
            for desc_name in descriptor_names:
                desc_func = getattr(Descriptors, desc_name)
                try:
                    value = desc_func(mol)
                    desc_values.append(value if not np.isnan(value) else 0)
                except:
                    desc_values.append(0)
            features.append(desc_values)
            valid_indices.append(idx)
    
    return np.array(features), valid_indices

def generate_ecfp6_features(smiles_list, radius=3, n_bits=2048):
    """Generate ECFP6 (Morgan) fingerprints with radius 3"""
    features = []
    valid_indices = []
    
    for idx, smiles in enumerate(smiles_list):
        mol = Chem.MolFromSmiles(smiles)
        if mol is not None:
            fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=radius, nBits=n_bits)
            features.append(np.array(fp))
            valid_indices.append(idx)
    
    return np.array(features), valid_indices

# ============================================================================
# Main Performance Impact Analysis
# ============================================================================

def run_performance_impact_analysis(
    data_dir='../data/exp_subset',
    output_dir='../data/performance',
    n_folds=5,
    n_replicates=3,
    random_seed=42
):
    """
    Run complete performance impact analysis
    
    Parameters:
    -----------
    data_dir : str
        Directory containing dataset parquet files
    output_dir : str
        Directory to save results (creates subdirectories per dataset)
    n_folds : int
        Number of cross-validation folds
    n_replicates : int
        Number of model replicates
    random_seed : int
        Random seed for reproducibility
    """
    
    # Create output directory
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    # Get all dataset files
    dataset_files = sorted(Path(data_dir).glob('*.parquet'))
    
    print(f"Found {len(dataset_files)} datasets to process")
    print("="*80)
    
    # Process each dataset
    for dataset_file in dataset_files[1:]:
        dataset_name = dataset_file.stem
        print(f"\n{'='*80}")
        print(f"Processing: {dataset_name}")
        print(f"{'='*80}")
        
        # Create dataset-specific output directory
        dataset_output_dir = Path(output_dir) / dataset_name
        dataset_output_dir.mkdir(parents=True, exist_ok=True)
        
        # Load dataset
        df = pd.read_parquet(dataset_file)
        target_col = dataset_name
        
        print(f"Dataset size: {len(df)} compounds")
        
        # Initialize results storage
        results_df = df.copy()
        
        # Descriptor types
        descriptor_types = ['avalon'] #, 'rdkit2d', 'ecfp6']
        
        # Training strategies
        training_strategies = ['non_aug', 'aug_plus_non_aug']
        
        # 5-Fold Cross Validation
        kf = KFold(n_splits=n_folds, shuffle=True, random_state=random_seed)
        
        for fold_idx, (train_idx, test_idx) in enumerate(kf.split(df)):
            print(f"\n--- Fold {fold_idx + 1}/{n_folds} ---")
            
            # Split data
            train_df = df.iloc[train_idx].copy()
            test_df = df.iloc[test_idx].copy()
            
            print(f"Train size: {len(train_df)}, Test size: {len(test_df)}")
            
            # Augment training data
            print("Augmenting training data...")
            try:
                augmentor = MMPDataAugmentor(
                    df=train_df,
                    smiles_col="SMILES",
                    target_cols=[target_col],
                    max_heavy=15,
                    max_ratio=0.4,
                    min_common=3,
                    pearson_thresh=0.3,
                    crmsd_thresh=1.0,
                    std_threshold=1.0,
                    verbose=False
                )
                
                augmented_train_df = augmentor.run()
                
                # Prepare augmented data for training
                aug_train_full = augmented_train_df.copy()
                aug_train_full['SMILES'] = aug_train_full['SMILES']
                aug_train_full[target_col] = aug_train_full[f'{target_col}_median']
                aug_train_full = aug_train_full[['SMILES', target_col, 'AUG']].dropna()
                
                print(f"Augmented train size: {len(aug_train_full)} (original: {len(train_df)})")
                
            except Exception as e:
                print(f"Warning: Augmentation failed for fold {fold_idx + 1}: {e}")
                aug_train_full = train_df.copy()
                aug_train_full['AUG'] = False
            
            # Prepare training sets
            train_non_aug = aug_train_full[aug_train_full['AUG'] == False].copy()
            train_aug_plus_non_aug = aug_train_full.copy()
            
            # For each descriptor type
            for descriptor_type in descriptor_types:
                print(f"\nDescriptor: {descriptor_type}")
                
                # Generate features
                if descriptor_type == 'avalon':
                    X_train_non_aug, valid_train_non_aug = generate_avalon_features(
                        train_non_aug['SMILES'].tolist()
                    )
                    X_train_aug, valid_train_aug = generate_avalon_features(
                        train_aug_plus_non_aug['SMILES'].tolist()
                    )
                    X_test, valid_test = generate_avalon_features(
                        test_df['SMILES'].tolist()
                    )
                    
                elif descriptor_type == 'rdkit2d':
                    X_train_non_aug, valid_train_non_aug = generate_rdkit2d_features(
                        train_non_aug['SMILES'].tolist()
                    )
                    X_train_aug, valid_train_aug = generate_rdkit2d_features(
                        train_aug_plus_non_aug['SMILES'].tolist()
                    )
                    X_test, valid_test = generate_rdkit2d_features(
                        test_df['SMILES'].tolist()
                    )
                    
                elif descriptor_type == 'ecfp6':
                    X_train_non_aug, valid_train_non_aug = generate_ecfp6_features(
                        train_non_aug['SMILES'].tolist()
                    )
                    X_train_aug, valid_train_aug = generate_ecfp6_features(
                        train_aug_plus_non_aug['SMILES'].tolist()
                    )
                    X_test, valid_test = generate_ecfp6_features(
                        test_df['SMILES'].tolist()
                    )
                
                # Get valid target values
                y_train_non_aug = train_non_aug[target_col].iloc[valid_train_non_aug].values
                y_train_aug = train_aug_plus_non_aug[target_col].iloc[valid_train_aug].values
                y_test = test_df[target_col].iloc[valid_test].values
                test_indices = test_df.index[valid_test]
                
                # For each replicate
                for replicate in range(n_replicates):
                    print(f"  Replicate {replicate + 1}/{n_replicates}...", end=' ')
                    
                    # Set seed for this replicate
                    rep_seed = random_seed + fold_idx * 100 + replicate
                    
                    # Train non-augmented model
                    model_non_aug = xgb.XGBRegressor(
                        n_estimators=100,
                        max_depth=20,
                        learning_rate=0.1,
                        random_state=rep_seed,
                        n_jobs=-1
                    )
                    model_non_aug.fit(X_train_non_aug, y_train_non_aug)
                    pred_non_aug = model_non_aug.predict(X_test)
                    
                    # Train augmented + non-augmented model
                    model_aug = xgb.XGBRegressor(
                        n_estimators=100,
                        max_depth=20,
                        learning_rate=0.1,
                        random_state=rep_seed,
                        n_jobs=-1
                    )
                    model_aug.fit(X_train_aug, y_train_aug)
                    pred_aug = model_aug.predict(X_test)
                    
                    # Store predictions
                    col_name_non_aug = f'pred_fold{fold_idx+1}_non_aug_{descriptor_type}_rep{replicate+1}'
                    col_name_aug = f'pred_fold{fold_idx+1}_aug_plus_non_aug_{descriptor_type}_rep{replicate+1}'
                    
                    # Initialize columns if needed
                    if col_name_non_aug not in results_df.columns:
                        results_df[col_name_non_aug] = np.nan
                    if col_name_aug not in results_df.columns:
                        results_df[col_name_aug] = np.nan
                    
                    # Store predictions at correct indices
                    results_df.loc[test_indices, col_name_non_aug] = pred_non_aug
                    results_df.loc[test_indices, col_name_aug] = pred_aug
                    
                    print("Done")
        
        # Save results
        output_file = dataset_output_dir / f'{dataset_name}_predictions.parquet'
        results_df.to_parquet(output_file, index=False)
        
        print(f"\n✓ Results saved to: {output_file}")
        print(f"  Total columns: {len(results_df.columns)}")
        print(f"  Prediction columns: {len([c for c in results_df.columns if c.startswith('pred_')])}")
    
    print(f"\n{'='*80}")
    print("All datasets processed successfully!")
    print(f"Results saved in: {output_dir}")

    
"""
MMP-Based Data Augmentation for Drug Discovery
A comprehensive class for expanding molecular datasets using Matched Molecular Pairs
"""

import os
import pandas as pd
import numpy as np
from rdkit import Chem, RDLogger
from rdkit.Chem import AllChem, inchi
import subprocess
import tempfile
import base64
import warnings
from tqdm import tqdm
from typing import List, Dict, Optional, Union
from collections import defaultdict
from itertools import combinations
from scipy.stats import pearsonr
import time


# Suppress RDKit warnings
RDLogger.DisableLog('rdApp.*')
warnings.filterwarnings("ignore", category=UserWarning, module="rdkit")
warnings.filterwarnings("ignore", category=FutureWarning, module="rdkit")
warnings.filterwarnings("ignore", category=DeprecationWarning, module="rdkit")


class MMPDataAugmentor:
    """
    Complete pipeline for MMP-based data augmentation.
    
    This class handles:
    1. Molecule fragmentation
    2. MMP identification
    3. Data augmentation through matched pairs
    4. Chemical structure generation
    5. Quality control and filtering
    
    Parameters
    ----------
    df : pd.DataFrame
        Input dataframe containing molecules and properties
    smiles_col : str
        Name of the column containing SMILES strings
    target_cols : Union[str, List[str]]
        Name(s) of target column(s) to augment
    mmpa_dir : str, default='./mmpa'
        Path to MMPA scripts directory
    symmetric : bool, default=True
        Generate symmetric MMPs (A->B and B->A)
    max_heavy : int, default=15
        Maximum heavy atom change allowed
    max_ratio : float, default=0.35
        Maximum ratio of change relative to molecule size
    min_common : int, default=4
        Minimum common MMPs required between scaffold pairs
    pearson_thresh : float, default=0.3
        Minimum Pearson correlation for scaffold pairing
    crmsd_thresh : float, default=0.8
        Maximum cRMSD for scaffold pairing
    std_threshold : float, default=0.8
        Maximum standard deviation for keeping augmented data
    verbose : bool, default=True
        Print progress messages
    """
    
    def __init__(
        self,
        df: pd.DataFrame,
        smiles_col: str = "SMILES",
        target_cols: Union[str, List[str]] = "Y",
        mmpa_dir: str = '../mmpa',
        symmetric: bool = True,
        max_heavy: int = 15,
        max_ratio: float = 0.4,
        min_common: int = 3,
        pearson_thresh: float = 0.3,
        crmsd_thresh: float = 1,
        std_threshold: float = 1,
        verbose: bool = True
    ):
        self.df_original = df.copy()
        self.smiles_col = smiles_col
        self.target_cols = [target_cols] if isinstance(target_cols, str) else target_cols
        self.mmpa_dir = mmpa_dir
        self.symmetric = symmetric
        self.max_heavy = max_heavy
        self.max_ratio = max_ratio
        self.min_common = min_common
        self.pearson_thresh = pearson_thresh
        self.crmsd_thresh = crmsd_thresh
        self.std_threshold = std_threshold
        self.verbose = verbose
        
        # Initialize storage for results
        self.augmented_df = None
        self.statistics = {}
        
        # Validate inputs
        self._validate_inputs()
    
    def _validate_inputs(self):
        """Validate input parameters and data"""
        if self.smiles_col not in self.df_original.columns:
            raise ValueError(f"SMILES column '{self.smiles_col}' not found in dataframe")
        
        for col in self.target_cols:
            if col not in self.df_original.columns:
                raise ValueError(f"Target column '{col}' not found in dataframe")
        
        if not os.path.exists(self.mmpa_dir):
            raise ValueError(f"MMPA directory '{self.mmpa_dir}' not found")
    
    def _log(self, message: str):
        """Print message if verbose mode is enabled"""
        if self.verbose:
            print(f"[MMPAugmentor] {message}")
    
    @staticmethod
    def _encode_string(s: str) -> str:
        """Encode string to base64"""
        return base64.urlsafe_b64encode(s.encode()).decode()
    
    @staticmethod
    def _smiles_to_inchikey(smiles: str) -> Optional[str]:
        """Convert SMILES to InChIKey"""
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return None
        try:
            return inchi.MolToInchiKey(mol)
        except:
            return None
    
    def _fragment_molecules(self, output_csv: str):
        """
        Fragment molecules using MMPA rfrag.py
        
        Parameters
        ----------
        output_csv : str
            Path to output CSV file
        """
        self._log("0) Generating molecular fragments...")
        
        # Prepare data for fragmentation
        df_prep = self.df_original[[self.smiles_col] + self.target_cols].copy()
        df_prep['ID'] = [self._smiles_to_inchikey(smi) for smi in df_prep[self.smiles_col]]
        
        # Create temporary directory
        with tempfile.TemporaryDirectory() as tmp:
            smi_path = os.path.join(tmp, 'input.smi')
            frag_path = os.path.join(tmp, 'fragmented.txt')
            mmps_path = os.path.join(tmp, 'mmps.csv')
            smirks_path = os.path.join(tmp, 'smirks.txt')
            cansmirks_path = os.path.join(tmp, 'cansmirks.txt')
            
            # Write SMILES file
            df_prep[[self.smiles_col, 'ID']].to_csv(
                smi_path, index=False, sep=' ', header=False
            )
            
            # Fragment molecules
            with open(frag_path, 'w') as out:
                subprocess.run(
                    ['python', f'{self.mmpa_dir}/rfrag.py'],
                    stdin=open(smi_path),
                    stdout=out
                )
            
            self._log("1) Indexing fragments and generating MMPs...")
            
            # Build MMPs
            cmd = ['python', f'{self.mmpa_dir}/indexing.py']
            if self.symmetric:
                cmd.append('-s')
            if self.max_heavy:
                cmd.extend(['-m', str(self.max_heavy)])
            if self.max_ratio:
                cmd.extend(['-r', str(self.max_ratio)])
            
            with open(mmps_path, 'w') as out:
                subprocess.run(cmd, stdin=open(frag_path), stdout=out)
            
            # Read and process MMPs
            with open(mmps_path) as f:
                lines = [line.strip() for line in f if line.strip()]
            
            splits = [line.split(',') for line in lines]
            df_mmps = pd.DataFrame(
                splits,
                columns=['L_SMILES', 'R_SMILES', 'L_ID', 'R_ID', 'SMIRKS', 'CORE']
            )
            
            # Map target values
            for target_col in self.target_cols:
                y_map = df_prep.set_index('ID')[target_col].to_dict()
                df_mmps[f'L_{target_col}'] = df_mmps['L_ID'].map(y_map)
                df_mmps[f'R_{target_col}'] = df_mmps['R_ID'].map(y_map)
                df_mmps[f'Delta_{target_col}'] = df_mmps[f'R_{target_col}'] - df_mmps[f'L_{target_col}']
            
            # Filter valid SMIRKS
            df_mmps = df_mmps[df_mmps['SMIRKS'].apply(
                lambda x: isinstance(x, str) and '>>' in x
            )]
            
            self._log("2) Canonicalizing SMIRKS...")
            
            # Canonicalize SMIRKS
            df_mmps['__row'] = range(len(df_mmps))
            df_mmps[['SMIRKS', '__row']].to_csv(
                smirks_path, index=False, sep=' ', header=False
            )
            
            with open(cansmirks_path, 'w') as out:
                subprocess.run(
                    ['python', f'{self.mmpa_dir}/cansmirk.py'],
                    stdin=open(smirks_path),
                    stdout=out
                )
            
            canon_df = pd.read_csv(
                cansmirks_path, sep=' ', names=['Canonical_SMIRKS', 'index']
            )
            
            df_mmps = df_mmps.merge(
                canon_df, left_on='__row', right_on='index'
            ).drop(columns=['__row', 'index'])
            
            # Split canonical SMIRKS
            df_mmps[['L_sub', 'R_sub']] = df_mmps['Canonical_SMIRKS'].str.split(
                '>>', expand=True
            )
            
            # Add encoded IDs
            df_mmps['L_sub_ID'] = [self._encode_string(k) for k in df_mmps['L_sub']]
            df_mmps['R_sub_ID'] = [self._encode_string(k) for k in df_mmps['R_sub']]
            df_mmps['SMIRKS_ID'] = [self._encode_string(k) for k in df_mmps['Canonical_SMIRKS']]
            df_mmps['CORE_ID'] = [self._encode_string(k) for k in df_mmps['CORE']]
            
            df_mmps = df_mmps.drop_duplicates()
            
            # Save to output
            df_mmps.to_csv(output_csv, index=False)
        
        return df_mmps
    
    def _augment_data(self, df_mmps: pd.DataFrame) -> pd.DataFrame:
        """
        Augment data using matched molecular pairs
        
        Parameters
        ----------
        df_mmps : pd.DataFrame
            DataFrame containing MMPs
            
        Returns
        -------
        pd.DataFrame
            Augmented dataset
        """
        self._log("3) Computing pairwise scaffold correlations...")
        
        # Group by CORE
        series = {core: group for core, group in df_mmps.groupby("CORE")}
        
        # Compute pairwise scores
        pair_scores = []
        series_items = list(series.items())
        total_combinations = len(series_items) * (len(series_items) - 1) // 2
        
        for (core1, df1), (core2, df2) in tqdm(
            combinations(series_items, 2),
            total=total_combinations,
            desc="Computing correlations",
            disable=not self.verbose
        ):
            subs1 = set(df1["L_sub"])
            subs2 = set(df2["L_sub"])
            common = subs1 & subs2
            
            if len(common) < self.min_common:
                continue
            
            merged = pd.merge(
                df1, df2,
                left_on=["L_sub", "R_sub"],
                right_on=["L_sub", "R_sub"],
                suffixes=('_1', '_2')
            )
            
            if len(merged) < self.min_common:
                continue
            
            # Use first target column for filtering
            target_col = self.target_cols[0]
            y1 = merged[f'Delta_{target_col}_1'].values
            y2 = merged[f'Delta_{target_col}_2'].values
            crmsd = np.sqrt(np.mean((y1 - y2) ** 2))
            
            try:
                corr = pearsonr(y1, y2)[0]
            except:
                corr = np.nan
            
            pair_scores.append((core1, core2, crmsd, corr, len(merged)))
        
        # Filter pairs
        filtered_pairs = [
            (s1, s2) for s1, s2, rmsd, corr, n in pair_scores
            if rmsd <= self.crmsd_thresh and (not np.isnan(corr) and corr >= self.pearson_thresh)
        ]
        
        self._log(f"4) Found {len(filtered_pairs)} valid scaffold pairs. Generating augmented data...")
        
        # Augment data
        augmented_entries = []
        
        for s1, s2 in tqdm(filtered_pairs, desc="Augmenting", disable=not self.verbose):
            df1 = series[s1]
            df2 = series[s2]
            
            # Create dictionaries for fast lookup
            df1_dict = defaultdict(list)
            for _, row in df1.iterrows():
                df1_dict[row["L_sub"]].append(row.to_dict())
            
            df2_dict = defaultdict(list)
            for _, row in df2.iterrows():
                df2_dict[row["L_sub"]].append(row.to_dict())
            
            # Generate augmented entries
            for target_col in self.target_cols:
                tf1 = df1[["L_sub", "R_sub", f"Delta_{target_col}"]].to_dict("records")
                tf2 = df2[["L_sub", "R_sub", f"Delta_{target_col}"]].to_dict("records")
                
                for entry in tf1:
                    l_sub = entry["L_sub"]
                    for base in df2_dict.get(l_sub, []):
                        r_sub = entry["R_sub"]
                        delta = entry[f"Delta_{target_col}"]
                        new_y = base[f"L_{target_col}"] + delta
                        smirks_new = l_sub + ">>" + r_sub
                        
                        aug_entry = {
                            "CORE": s1,
                            "L_sub": l_sub,
                            "R_sub": r_sub,
                            f"L_{target_col}": base[f"L_{target_col}"],
                            f"R_{target_col}": new_y,
                            f"Delta_{target_col}": delta,
                            "AUG": True,
                            "L_SMILES": base.get("L_SMILES"),
                            "L_ID": base.get("L_ID"),
                            "L_sub_ID": base.get("L_sub_ID"),
                            "R_sub_ID": self._encode_string(r_sub),
                            "SMIRKS": smirks_new,
                            "SMIRKS_ID": self._encode_string(smirks_new),
                            "CORE_ID": self._encode_string(s1)
                        }
                        augmented_entries.append(aug_entry)
                
                for entry in tf2:
                    l_sub = entry["L_sub"]
                    for base in df1_dict.get(l_sub, []):
                        r_sub = entry["R_sub"]
                        delta = entry[f"Delta_{target_col}"]
                        new_y = base[f"L_{target_col}"] + delta
                        smirks_new = l_sub + ">>" + r_sub
                        
                        aug_entry = {
                            "CORE": s2,
                            "L_sub": l_sub,
                            "R_sub": r_sub,
                            f"L_{target_col}": base[f"L_{target_col}"],
                            f"R_{target_col}": new_y,
                            f"Delta_{target_col}": delta,
                            "AUG": True,
                            "L_SMILES": base.get("L_SMILES"),
                            "L_ID": base.get("L_ID"),
                            "L_sub_ID": base.get("L_sub_ID"),
                            "R_sub_ID": self._encode_string(r_sub),
                            "SMIRKS": smirks_new,
                            "SMIRKS_ID": self._encode_string(smirks_new),
                            "CORE_ID": self._encode_string(s2)
                        }
                        augmented_entries.append(aug_entry)
        
        augmented_df = pd.DataFrame(augmented_entries)
        
        # Mark original data
        df_mmps["AUG"] = False
        
        # Combine original and augmented
        combined = pd.concat([df_mmps, augmented_df], ignore_index=True)
        
        return combined
    
    def _apply_transformations(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Apply chemical transformations to generate R_SMILES
        
        Parameters
        ----------
        df : pd.DataFrame
            DataFrame with augmented data
            
        Returns
        -------
        pd.DataFrame
            DataFrame with applied transformations
        """
        self._log("5) Applying chemical transformations...")
        
        df = df.drop_duplicates()
        mask = df["AUG"] == True
        indices = df[mask].index
        
        rxn_cache = {}
        heavy_cache = {}
        core_cache = {}
        
        new_rows = []
        failure_tracker = {"total": 0, "empty_prodsets": 0}
        
        for idx in tqdm(indices, desc="Transforming", disable=not self.verbose):
            row = df.loc[idx]
            l_smiles = row["L_SMILES"]
            smirks = row["SMIRKS"]
            core = row["CORE"]
            
            r_smiles_list = self._fast_apply_transformation(
                smirks, l_smiles, rxn_cache, core_cache, heavy_cache, core,
                failure_tracker
            )
            
            if r_smiles_list:
                for r_smiles in r_smiles_list:
                    mol = Chem.MolFromSmiles(r_smiles)
                    if mol:
                        r_id = Chem.InchiToInchiKey(Chem.MolToInchi(mol))
                        new_row = row.copy()
                        new_row["R_SMILES"] = r_smiles
                        new_row["R_ID"] = r_id
                        new_rows.append(new_row)
        
        df_non_aug = df[~mask]
        df_aug_expanded = pd.DataFrame(new_rows)
        df_final = pd.concat([df_non_aug, df_aug_expanded], ignore_index=True)
        
        # Report failure rate
        total = failure_tracker["total"]
        failed = failure_tracker["empty_prodsets"]
        if total > 0:
            self._log(
                f"⚠️  Empty product sets in {failed}/{total} "
                f"({100*failed/total:.2f}%) transformations"
            )
        
        return df_final
    
    @staticmethod
    def _fast_apply_transformation(transformation, l_smiles, rxn_cache, 
                                   core_cache, heavy_cache, core_smarts,
                                   failure_tracker=None):
        """Apply SMIRKS transformation with caching"""
        if pd.isna(transformation) or pd.isna(l_smiles):
            return None
        
        # Cache reaction
        if transformation not in rxn_cache:
            try:
                rxn = AllChem.ReactionFromSmarts(transformation)
                left_smi, right_smi = transformation.split(">>")
                left_mol = Chem.MolFromSmarts(left_smi)
                right_mol = Chem.MolFromSmarts(right_smi)
                delta_heavy = right_mol.GetNumHeavyAtoms() - left_mol.GetNumHeavyAtoms()
                rxn_cache[transformation] = (rxn, delta_heavy)
            except:
                return None
        else:
            rxn, delta_heavy = rxn_cache[transformation]
        
        # Cache molecule
        if l_smiles not in heavy_cache:
            mol_l = Chem.MolFromSmiles(l_smiles)
            if mol_l is None:
                return None
            n_heavy_l = mol_l.GetNumHeavyAtoms()
            heavy_cache[l_smiles] = (mol_l, n_heavy_l)
        else:
            mol_l, n_heavy_l = heavy_cache[l_smiles]
        
        # Cache core
        if core_smarts not in core_cache:
            core_mol = Chem.MolFromSmarts(core_smarts)
            if core_mol is None:
                return None
            core_cache[core_smarts] = core_mol
        else:
            core_mol = core_cache[core_smarts]
        
        # Run reaction
        try:
            products = rxn.RunReactants((mol_l,))
        except:
            return None
        
        if failure_tracker is not None:
            failure_tracker["total"] += 1
            if not products:
                failure_tracker["empty_prodsets"] += 1
        
        all_products = []
        for prod_set in products:
            for prod in prod_set:
                if prod is None:
                    continue
                try:
                    if not prod.HasSubstructMatch(core_mol):
                        continue
                except:
                    continue
                n_heavy_r = prod.GetNumHeavyAtoms()
                if n_heavy_r - n_heavy_l != delta_heavy:
                    continue
                all_products.append(Chem.MolToSmiles(prod, isomericSmiles=True))
        
        return all_products if all_products else None
    
    def _prepare_output(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Prepare final output with proper formatting and statistics
        
        Parameters
        ----------
        df : pd.DataFrame
            Processed dataframe
            
        Returns
        -------
        pd.DataFrame
            Final output dataframe
        """
        self._log("6) Preparing output...")
        
        # Extract L and R entries
        output_rows = []
        
        for target_col in self.target_cols:
            l_df = df[["L_SMILES", f"L_{target_col}", "AUG"]].copy()
            l_df.columns = ["SMILES", target_col, "AUG"]
            
            r_df = df[["R_SMILES", f"R_{target_col}", "AUG"]].copy()
            r_df.columns = ["SMILES", target_col, "AUG"]
            
            combined = pd.concat([l_df, r_df])
            output_rows.append(combined)
        
        # Combine all targets
        clean_df = pd.concat(output_rows, axis=0)
        
        # Remove invalid SMILES
        total_before = len(clean_df)
        clean_df = clean_df.dropna(subset=["SMILES"])
        clean_df = clean_df[clean_df["SMILES"].apply(
            lambda x: Chem.MolFromSmiles(x) is not None
        )]
        total_after = len(clean_df)
        
        fail_pct = 100 * (total_before - total_after) / total_before
        self._log(
            f"Invalid SMILES removed: {total_before - total_after}/{total_before} "
            f"({fail_pct:.2f}%)"
        )
        
        # Standardize SMILES
        self._log("7) Standardizing SMILES...")
        tqdm.pandas(desc="Standardizing", disable=not self.verbose)
        clean_df["SMILES"] = clean_df["SMILES"].progress_apply(
            lambda x: Chem.MolToSmiles(Chem.MolFromSmiles(x), isomericSmiles=True)
        )
        
        # Generate InChIKeys
        self._log("8) Generating InChIKeys...")
        tqdm.pandas(desc="InChIKeys", disable=not self.verbose)
        clean_df['InChIKey'] = clean_df["SMILES"].progress_apply(self._smiles_to_inchikey)
        
        # Separate experimental and predicted
        clean_df_exp = clean_df[clean_df["AUG"] != True].copy()
        clean_df_pred = clean_df[clean_df["AUG"] == True].copy()
        
        # Aggregate predicted values
        output_dfs = []
        for target_col in self.target_cols:
            grouped_pred = clean_df_pred.groupby("InChIKey")[target_col].agg(
                ["median", "std", "mean", "count"]
            ).reset_index()
            grouped_pred.columns = [
                "InChIKey", 
                f"{target_col}_median",
                f"{target_col}_std", 
                f"{target_col}_mean",
                f"{target_col}_count"
            ]
            grouped_pred["AUG"] = True
            
            # Filter by std threshold
            grouped_pred = grouped_pred[
                grouped_pred[f"{target_col}_std"] <= self.std_threshold
            ]
            
            # Add SMILES
            inchikey_to_smiles = clean_df_pred.drop_duplicates("InChIKey").set_index(
                "InChIKey"
            )["SMILES"].to_dict()
            grouped_pred["SMILES"] = grouped_pred["InChIKey"].map(inchikey_to_smiles)
            
            # Experimental data
            exp_out = clean_df_exp[['InChIKey', "SMILES", target_col]].copy()
            exp_out[f"{target_col}_median"] = exp_out[target_col]
            exp_out[f"{target_col}_std"] = None
            exp_out[f"{target_col}_mean"] = exp_out[target_col]
            exp_out[f"{target_col}_count"] = 1
            exp_out["AUG"] = False
            exp_out = exp_out.drop(columns=[target_col])
            
            # Remove predicted entries that exist in experimental
            known_keys = set(exp_out['InChIKey'])
            grouped_pred = grouped_pred[~grouped_pred['InChIKey'].isin(known_keys)]
            
            # Combine
            combined = pd.concat([
                exp_out.drop_duplicates("InChIKey"),
                grouped_pred.drop_duplicates("InChIKey")
            ], ignore_index=True)
            
            output_dfs.append(combined)
        
        # Merge all targets
        final_df = output_dfs[0]
        for df_target in output_dfs[1:]:
            final_df = final_df.merge(df_target, on=["InChIKey", "SMILES", "AUG"], how="outer")
        
        # Calculate statistics
        self.statistics = {
            "n_original": len(clean_df_exp.drop_duplicates("InChIKey")),
            "n_augmented": len(grouped_pred.drop_duplicates("InChIKey")),
            "n_total": len(final_df),
            "augmentation_ratio": len(grouped_pred) / len(clean_df_exp) if len(clean_df_exp) > 0 else 0
        }
        
        self._log(f"✅ Augmentation complete!")
        self._log(f"   Original compounds: {self.statistics['n_original']}")
        self._log(f"   Augmented compounds: {self.statistics['n_augmented']}")
        self._log(f"   Total compounds: {self.statistics['n_total']}")
        self._log(f"   Augmentation ratio: {self.statistics['augmentation_ratio']:.2f}x")
        
        return final_df
    
    def run(self, output_path: Optional[str] = None) -> pd.DataFrame:
        """
        Run the complete augmentation pipeline
        
        Parameters
        ----------
        output_path : str, optional
            Path to save the output CSV/parquet file
            
        Returns
        -------
        pd.DataFrame
            Augmented dataset with statistics
        """
        start_time = time.time()
        
        # Create temporary output file
        with tempfile.NamedTemporaryFile(
            mode='w', suffix='.csv', delete=False
        ) as tmp_file:
            tmp_output = tmp_file.name
        
        try:
            # Step 1: Fragment and generate MMPs
            df_mmps = self._fragment_molecules(tmp_output)
            
            # Step 2: Augment data
            df_augmented = self._augment_data(df_mmps)
            
            # Step 3: Apply transformations
            df_transformed = self._apply_transformations(df_augmented)
            
            # Step 4: Prepare output
            self.augmented_df = self._prepare_output(df_transformed)
            
            # Save if path provided
            if output_path:
                if output_path.endswith('.parquet'):
                    self.augmented_df.to_parquet(output_path, index=False)
                else:
                    self.augmented_df.to_csv(output_path, index=False)
                self._log(f"Output saved to: {output_path}")
            
        finally:
            # Cleanup temporary file
            if os.path.exists(tmp_output):
                os.unlink(tmp_output)
        
        elapsed = time.time() - start_time
        self._log(f"⏱️  Total time: {elapsed:.2f} seconds")
        
        # Get InChIKeys from augmented_df
        augmented_inchikeys = set(self.augmented_df['InChIKey'].dropna())

        # Get InChIKeys from original
        self.df_original['InChIKey'] = self.df_original[self.smiles_col].apply(self._smiles_to_inchikey)
        original_inchikeys = set(self.df_original['InChIKey'].dropna())

        # Find missing compounds
        missing_inchikeys = original_inchikeys - augmented_inchikeys

        if len(missing_inchikeys) > 0:
            self._log(f"Adding {len(missing_inchikeys)} missing original compounds...")

            # Get missing compounds from original
            missing_df = self.df_original[self.df_original['InChIKey'].isin(missing_inchikeys)].copy()

            # Format to match augmented_df structure
            missing_formatted = pd.DataFrame()
            missing_formatted['InChIKey'] = missing_df['InChIKey']
            missing_formatted['SMILES'] = missing_df[self.smiles_col]
            missing_formatted['AUG'] = False

            # Add target columns
            for target_col in self.target_cols:
                missing_formatted[f'{target_col}_median'] = missing_df[target_col]
                missing_formatted[f'{target_col}_mean'] = missing_df[target_col]
                missing_formatted[f'{target_col}_std'] = None
                missing_formatted[f'{target_col}_count'] = 1

            # Concatenate
            self.augmented_df = pd.concat([self.augmented_df, missing_formatted], ignore_index=True)

            # Update statistics
            self.statistics['n_original'] = len(original_inchikeys)        
        
        return self.augmented_df
    
    def get_statistics(self) -> Dict:
        """Return augmentation statistics"""
        return self.statistics
    
    def get_experimental_only(self) -> pd.DataFrame:
        """Return only experimental compounds"""
        if self.augmented_df is None:
            raise ValueError("Run augmentation first using .run()")
        return self.augmented_df[self.augmented_df["AUG"] == False].copy()
    
    def get_augmented_only(self) -> pd.DataFrame:
        """Return only augmented compounds"""
        if self.augmented_df is None:
            raise ValueError("Run augmentation first using .run()")
        return self.augmented_df[self.augmented_df["AUG"] == True].copy()



In [ ]:
run_performance_impact_analysis(
    data_dir='../data/exp_subset',
    output_dir='../data/performance',
    n_folds=5,
    n_replicates=1,
    random_seed=42
)

Found 31 datasets to process

Processing: Clearance Microsomal (Mouse)
Dataset size: 3797 compounds

--- Fold 1/5 ---
Train size: 3037, Test size: 760
Augmenting training data...


[19:14:13] WARNING: not removing hydrogen atom with dummy atom neighbors
[19:14:13] WARNING: not removing hydrogen atom with dummy atom neighbors
[19:14:24] WARNING: not removing hydrogen atom with dummy atom neighbors
[19:14:24] WARNING: not removing hydrogen atom with dummy atom neighbors
[19:14:25] WARNING: not removing hydrogen atom with dummy atom neighbors
[19:14:25] WARNING: not removing hydrogen atom with dummy atom neighbors
[19:14:25] WARNING: not removing hydrogen atom with dummy atom neighbors
[19:14:25] WARNING: not removing hydrogen atom with dummy atom neighbors
[19:14:25] WARNING: not removing hydrogen atom with dummy atom neighbors
[19:14:25] WARNING: not removing hydrogen atom with dummy atom neighbors
[19:14:25] WARNING: not removing hydrogen atom with dummy atom neighbors
[19:14:25] WARNING: not removing hydrogen atom with dummy atom neighbors
[19:14:25] WARNING: not removing hydrogen atom with dummy atom neighbors
[19:14:25] WARNING: not removing hydrogen atom with

Augmented train size: 3404 (original: 3037)

Descriptor: avalon
  Replicate 1/1... Done

--- Fold 2/5 ---
Train size: 3037, Test size: 760
Augmenting training data...


[19:35:35] WARNING: not removing hydrogen atom with dummy atom neighbors
[19:35:35] WARNING: not removing hydrogen atom with dummy atom neighbors
[19:35:35] WARNING: not removing hydrogen atom with dummy atom neighbors
[19:35:35] WARNING: not removing hydrogen atom with dummy atom neighbors
[19:35:49] WARNING: not removing hydrogen atom with dummy atom neighbors
[19:35:49] WARNING: not removing hydrogen atom with dummy atom neighbors
[19:35:49] WARNING: not removing hydrogen atom with dummy atom neighbors
[19:35:49] WARNING: not removing hydrogen atom with dummy atom neighbors
[19:35:57] WARNING: not removing hydrogen atom with dummy atom neighbors
[19:35:57] WARNING: not removing hydrogen atom with dummy atom neighbors
[19:36:01] WARNING: not removing hydrogen atom with dummy atom neighbors
[19:36:01] WARNING: not removing hydrogen atom with dummy atom neighbors
[19:36:01] WARNING: not removing hydrogen atom with dummy atom neighbors
[19:36:01] WARNING: not removing hydrogen atom with

Augmented train size: 3404 (original: 3037)

Descriptor: avalon
  Replicate 1/1... Done

--- Fold 3/5 ---
Train size: 3038, Test size: 759
Augmenting training data...


...

[05:08:42] WARNING: not removing hydrogen atom with dummy atom neighbors
[05:08:42] WARNING: not removing hydrogen atom with dummy atom neighbors
[05:08:42] WARNING: not removing hydrogen atom with dummy atom neighbors
[05:08:42] WARNING: not removing hydrogen atom with dummy atom neighbors
[05:08:44] WARNING: not removing hydrogen atom with dummy atom neighbors
[05:08:44] WARNING: not removing hydrogen atom with dummy atom neighbors
[05:08:44] WARNING: not removing hydrogen atom with dummy atom neighbors
[05:08:44] WARNING: not removing hydrogen atom with dummy atom neighbors
[05:08:44] WARNING: not removing hydrogen atom with dummy atom neighbors
[05:08:44] WARNING: not removing hydrogen atom with dummy atom neighbors
[05:08:45] WARNING: not removing hydrogen atom with dummy atom neighbors
[05:08:45] WARNING: not removing hydrogen atom with dummy atom neighbors
[05:08:45] WARNING: not removing hydrogen atom with dummy atom neighbors
[05:08:45] WARNING: not removing hydrogen atom with

Augmented train size: 865 (original: 861)

Descriptor: avalon
  Replicate 1/1... Done

--- Fold 5/5 ---
Train size: 861, Test size: 215
Augmenting training data...


[05:15:23] WARNING: not removing hydrogen atom with dummy atom neighbors
[05:15:23] WARNING: not removing hydrogen atom with dummy atom neighbors
[05:15:24] WARNING: not removing hydrogen atom with dummy atom neighbors
[05:15:24] WARNING: not removing hydrogen atom with dummy atom neighbors
[05:15:24] WARNING: not removing hydrogen atom with dummy atom neighbors
[05:15:24] WARNING: not removing hydrogen atom with dummy atom neighbors
[05:15:24] WARNING: not removing hydrogen atom with dummy atom neighbors
[05:15:24] WARNING: not removing hydrogen atom with dummy atom neighbors
[05:15:24] WARNING: not removing hydrogen atom with dummy atom neighbors
[05:15:24] WARNING: not removing hydrogen atom with dummy atom neighbors
[05:15:24] WARNING: not removing hydrogen atom with dummy atom neighbors
[05:15:24] WARNING: not removing hydrogen atom with dummy atom neighbors
[05:15:24] WARNING: not removing hydrogen atom with dummy atom neighbors
[05:15:24] WARNING: not removing hydrogen atom with

Augmented train size: 881 (original: 861)

Descriptor: avalon
  Replicate 1/1... Done

✓ Results saved to: ../data/performance/Half Life (Rat, Plasma)/Half Life (Rat, Plasma)_predictions.parquet
  Total columns: 12
  Prediction columns: 10

Processing: LD50 (Rat, po)
Dataset size: 6590 compounds

--- Fold 1/5 ---
Train size: 5272, Test size: 1318
Augmenting training data...


[06:16:38] WARNING: not removing hydrogen atom with dummy atom neighbors
[06:16:38] WARNING: not removing hydrogen atom with dummy atom neighbors
[06:44:50] WARNING: not removing hydrogen atom with dummy atom neighbors
[06:44:50] WARNING: not removing hydrogen atom with dummy atom neighbors
[06:52:55] WARNING: not removing hydrogen atom with dummy atom neighbors
[06:52:55] WARNING: not removing hydrogen atom with dummy atom neighbors
[06:52:55] WARNING: not removing hydrogen atom with dummy atom neighbors
[06:52:55] WARNING: not removing hydrogen atom with dummy atom neighbors
[06:54:11] WARNING: not removing hydrogen atom with dummy atom neighbors
[06:54:11] WARNING: not removing hydrogen atom with dummy atom neighbors
